# Import packages

In [2]:
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))

from common import *

# 1. Load dataset

In [3]:
train_data = pd.read_csv(join(PATH.input, "train_data.csv"))
sample_submission = pd.read_csv(join(PATH.input, "sample_submission.csv"))
train_data = train_data.fillna(method = 'bfill')
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3859200 entries, 0 to 3859199
Data columns (total 13 columns):
 #   Column   Dtype  
---  ------   -----  
 0   TurbID   int64  
 1   Day      int64  
 2   Tmstamp  object 
 3   Wspd     float64
 4   Wdir     float64
 5   Etmp     float64
 6   Itmp     float64
 7   Ndir     float64
 8   Pab1     float64
 9   Pab2     float64
 10  Pab3     float64
 11  Prtv     float64
 12  Patv     float64
dtypes: float64(10), int64(2), object(1)
memory usage: 382.8+ MB


# 2. Preprocessing

In [4]:
tms_list = list(pd.unique(train_data['Tmstamp']))
train_data['Tmstamp'] = train_data['Tmstamp'].apply(lambda x: tms_list.index(x)+1)

In [5]:
SEQ_LEN = 5
(train_x, train_y), (val_x, val_y), (test_x, test_y) = make_train_val_test_data_single_turbine(train_data, SEQ_LEN)

# 3. Modeling

In [12]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU

tf.keras.utils.set_random_seed(RANDOM_STATE)
for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

2022-07-11 07:31:55.746368: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-11 07:31:55.749841: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-11 07:31:55.750006: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [24]:
tids = train_data['TurbID'].unique()

In [22]:
models = []
for turbID in tqdm(tids):
    model = Sequential([
        GRU(256, input_shape=train_x[turbID-1][0].shape),  # TurbID startswith 1, data, model startswith 0
        Dense(516, activation='relu'),
        Dense(288, activation='relu')
    ])
    optimizer = tf.optimizers.RMSprop(0.001)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    models.append(model)

100%|██████████| 134/134 [00:18<00:00,  7.13it/s]


# 4. Training

In [27]:
from tensorflow.keras.callbacks import EarlyStopping

for model, tx, ty, vx, vy in tqdm(zip(models, train_x, train_y, val_x, val_y)):
    model.fit(tx, ty, validation_data=(vx, vy), epochs=1000, batch_size=128, callbacks=[EarlyStopping(patience=10, restore_best_weights=True)]);

0it [00:00, ?it/s]

Epoch 1/1000
2/2 [==============================] - 1s 328ms/step - loss: 493142.0000 - mae: 465.9098 - val_loss: 721141.0000 - val_mae: 628.0217
Epoch 2/1000
2/2 [==============================] - 0s 65ms/step - loss: 490218.8750 - mae: 464.5632 - val_loss: 711819.2500 - val_mae: 623.6963
Epoch 3/1000
2/2 [==============================] - 0s 61ms/step - loss: 482959.7812 - mae: 461.3134 - val_loss: 696852.1875 - val_mae: 616.8702
Epoch 4/1000
2/2 [==============================] - 0s 69ms/step - loss: 471726.0938 - mae: 456.3490 - val_loss: 678096.8125 - val_mae: 608.4957
Epoch 5/1000
2/2 [==============================] - 0s 73ms/step - loss: 458464.0938 - mae: 450.8914 - val_loss: 657877.2500 - val_mae: 600.1135
Epoch 6/1000
2/2 [==============================] - 0s 63ms/step - loss: 443943.4688 - mae: 444.9218 - val_loss: 635974.3125 - val_mae: 591.5101
Epoch 7/1000
2/2 [==============================] - 0s 70ms/step - loss: 428669.1562 - mae: 439.1902 - val_loss: 614822.6875 - va

2/2 [==============================] - 0s 70ms/step - loss: 237662.5156 - mae: 403.1352 - val_loss: 319123.5000 - val_mae: 510.0566
Epoch 58/1000
2/2 [==============================] - 0s 61ms/step - loss: 235999.7031 - mae: 400.5259 - val_loss: 315437.6250 - val_mae: 507.1244
Epoch 59/1000
2/2 [==============================] - 0s 66ms/step - loss: 237761.8125 - mae: 396.8134 - val_loss: 318971.1250 - val_mae: 509.8693


1it [00:07,  7.66s/it]

Epoch 1/1000
2/2 [==============================] - 1s 284ms/step - loss: 358991.4062 - mae: 393.4587 - val_loss: 614853.8750 - val_mae: 509.3658
Epoch 2/1000
2/2 [==============================] - 0s 66ms/step - loss: 356276.0000 - mae: 392.0320 - val_loss: 606657.7500 - val_mae: 506.2169
Epoch 3/1000
2/2 [==============================] - 0s 65ms/step - loss: 349582.3438 - mae: 388.6045 - val_loss: 593832.0000 - val_mae: 501.4028
Epoch 4/1000
2/2 [==============================] - 0s 63ms/step - loss: 339570.8750 - mae: 383.4772 - val_loss: 578596.0625 - val_mae: 496.1324
Epoch 5/1000
2/2 [==============================] - 0s 64ms/step - loss: 327838.4688 - mae: 377.8191 - val_loss: 561524.1875 - val_mae: 491.0749
Epoch 6/1000
2/2 [==============================] - 0s 63ms/step - loss: 315391.2188 - mae: 372.1351 - val_loss: 544052.2500 - val_mae: 486.9051
Epoch 7/1000
2/2 [==============================] - 0s 67ms/step - loss: 302664.7188 - mae: 366.7054 - val_loss: 527528.4375 - va

...

2/2 [==============================] - 0s 64ms/step - loss: 159566.3281 - mae: 317.6860 - val_loss: 211193.0781 - val_mae: 369.3812
Epoch 58/1000
2/2 [==============================] - 0s 73ms/step - loss: 159270.8750 - mae: 316.8754 - val_loss: 211421.5469 - val_mae: 358.5374


133it [22:11,  9.17s/it]

Epoch 1/1000


133it [22:11, 10.01s/it]


TypeError: 'NoneType' object is not callable

# 5. Generate submission

In [59]:
for tid in tids:
    sample_submission.loc[sample_submission['TurbID'] == tid, 'Patv'] = models[tid-1].predict(test_x[tid-1]).reshape(-1)
sample_submission.to_csv(join(PATH.output, "proposed4.csv"), index=False)

1/1 [==============================] - 0s 21ms/step
